## 환경설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/딥러닝/감성분석')

In [ ]:
# 파일 압축 풀기
import zipfile
         
fantasy_zip = zipfile.ZipFile('/content/drive/My Drive/딥러닝/감성분석/2020-1-deeplearning-final.zip')
fantasy_zip.extractall('/content/drive/My Drive/딥러닝/감성분석')
 
fantasy_zip.close()

In [ ]:
# 패키지 임포트

import pandas as pd
!pip install konlpy
from konlpy.tag import Okt
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM, SpatialDropout1D, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


## 데이터

In [ ]:
# loading data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,id,rating,raw,chr_token,nn_token
0,R024795,1,1982년도에 이런영화가 제작이 됐다는거 자체가 놀라운거라고. 재미없다고 씨부리는것...,1 9 8 2 년 도 에 이 런 영 화 가 제 작 이 됐 다 는 거 자 체 가 놀 ...,년도 영화 제작 거 자체 거 씨 것 엄매 뱃속 정자 난자 전 리를리감 독 전구 에디...
1,R068332,1,핸드핼드기법을 적절히 써가며 현실과 환상의 대비를 확연하게 보여주는 영화,핸 드 핼 드 기 법 을 적 절 히 써 가 며 현 실 과 환 상 의 대 비 를 확 ...,핸드 핼드 기법 현실 환상 대비 영화
2,R027236,1,....엑스맨을 깔끔하게 정리를 했다는 자체가 이미 뛰어난 영화라는 것을 증명하고 있다,. . . . 엑 스 맨 을 깔 끔 하 게 정 리 를 했 다 는 자 체 가 이 미 ...,엑스맨 정리 자체 영화 것 증명
3,R050965,0,뭐 이런 쓰레기 영화가 다있냐 인물관계도 정리안해줘 줄거리도 없어 내용도 ㅉㅉ,뭐 이 런 쓰 레 기 영 화 가 다 있 냐 인 물 관 계 도 정 리 안 해 줘 줄 ...,쓰레기 영화 인물 관계 정리안 줄거리 내용 ㅉ
4,R016317,0,육식공룡중에서 빌딩보다 더큰공룡이어딧노 ㅋㅋㅋㅋㅋㅋ,육 식 공 룡 중 에 서 빌 딩 보 다 더 큰 공 룡 이 어 딧 노 ㅋ ㅋ ㅋ ㅋ ㅋ ㅋ,육식 공 룡 중 빌딩 공룡 어딧노 ㅋ ㅋ


In [ ]:
# class 분포 확인
train.groupby(['rating'])['id'].size()

rating
0    27255
1    27255
Name: id, dtype: int64

# SKT : 한글 pre-trained BERT(KOBERT)

In [ ]:
!pip install transformers # hugging face

     |████████████████████████████████| 675kB 3.5MB/s 
     |████████████████████████████████| 890kB 15.5MB/s 
     |████████████████████████████████| 3.8MB 23.5MB/s 
     |████████████████████████████████| 1.1MB 43.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=72926c347f28b82d48982a5c83f4473604d2320afae4536203c699c775068e15
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
test

,id,raw,chr_token,nn_token
0,R009132,감독이 넘 제멋대로 자유분방(?)하게 만든거같다는...보구나서 느낌은 모지 이영화?,감 독 이 넘 제 멋 대 로 자 유 분 방 ( ? ) 하 게 만 든 거 같 다 는 ...,감독 자유 분방 느낌 모지 이영화
1,R045659,제작사와감독에게할복을요청하고싶다...일본영화의 최대망작!!!,제 작 사 와 감 독 에 게 할 복 을 요 청 하 고 싶 다 . . . 일 본 영 ...,제작사 감독 할복 요청 일본 영화 최대망작
2,R015196,드라마는 재미 있고 모든 출연자들 자알합니다만 단 한 사람 추잡스러워서 드라마 질을...,드 라 마 는 재 미 있 고 모 든 출 연 자 들 자 알 합 니 다 만 단 한 사 ...,드라마 재미 출연자 자 사람 잡 드라마 질 입술 성형 처음 때 지금 잡 서리
3,R063595,초등생인 첫째아이가 어렸을때 좋아했던 토마스.. 다시봐도 재밌다네요..둘째도 너무 ...,초 등 생 인 첫 째 아 이 가 어 렸 을 때 좋 아 했 던 토 마 스 . . 다 ...,초등 아이 때 토마스 다 아이 영화
4,R053340,중국을 보고싶다면 미모와 연기력을 다 갖춘 중국의 아이콘을 보고싶다면...,중 국 을 보 고 싶 다 면 미 모 와 연 기 력 을 다 갖 춘 중 국 의 아 이 ...,중국 미모 연기력 중국 아이콘
...,...,...,...,...
13621,R002153,생각지도못한 반전영화 12년 전 영화인데도 최고,생 각 지 도 못 한 반 전 영 화 1 2 년 전 영 화 인 데 도 최 고,생각 전 영화 년 전 영화 최고
13622,R029129,결말이 예측 되다가도 완전 빗나간 것 같다가 어느정도 예상에 접근하며 끝난 영화. ...,결 말 이 예 측 되 다 가 도 완 전 빗 나 간 것 같 다 가 어 느 정 도 예 ...,결말 예측 완전 것 정도 예상 접근 영화 흥미 몰입도 상당 영화 현실 것 고 설명 ...
13623,R021090,여태것 본 공포영화 중 젤 쓰레기 내용은 좋은데 감정적으로 대응하는 부분이 쓸대없이...,여 태 것 본 공 포 영 화 중 젤 쓰 레 기 내 용 은 좋 은 데 감 정 적 으 ...,태 것 공포 영화 중 젤 쓰레기 내용 감정 대응 부분 대 번 볼영
13624,R013734,극장영화가 아니라 TV영화임! 이러한 2시간 영화를 찍을려고 해도 몇달동안 무더운 ...,극 장 영 화 가 아 니 라 T V 영 화 임 ! 이 러 한 2 시 간 영 화 를 ...,극장 영화 영화 시간 영화 달 동안 여름 밤 낮 새벽 잠 것 촬영 신인감독 신인 배...


### 버트 인풋 만들기

https://github.com/monologg/KoBERT-NER 에서 tokenizer 다운

In [ ]:
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        self.max_len_single_sentence = self.max_len - 2  # take into account special tokens
        self.max_len_sentences_pair = self.max_len - 3  # take into account special tokens

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 128, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A RoBERTa sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A BERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [ ]:
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

Setting 'max_len_single_sentence' is now deprecated. This value is automatically set up.
Setting 'max_len_sentences_pair' is now deprecated. This value is automatically set up.


In [ ]:
# 문장을 [토큰 인풋, 마스크 인풋, 세그먼트 인풋] 순으로 변환
# (bert input 형식으로 문장 변환)

def convert_data(data_df):
    global tokenizer
    
    SEQ_LEN = 64 #SEQ_LEN : 버트에 들어갈 인풋의 길이
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, pad_to_max_length=True)
       
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        segment = [0]*SEQ_LEN

        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        targets.append(data_df[LABEL_COLUMN][i])

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y


# 긍부정 문장을 포함하고 있는 칼럼
DATA_COLUMN = "raw"
# 긍정인지 부정인지를 (1=긍정,0=부정) 포함하고 있는 칼럼
LABEL_COLUMN = "rating"

# train 데이터를 버트 인풋에 맞게 변환
train_x, train_y = load_data(train)

100%|██████████| 54510/54510 [00:11<00:00, 4640.93it/s]


## 모델링

In [ ]:
SEQ_LEN = 64
model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)

token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')

bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

In [ ]:
bert_outputs

(<tf.Tensor 'tf_bert_model_3/Identity:0' shape=(None, 64, 768) dtype=float32>,
 <tf.Tensor 'tf_bert_model_3/Identity_1:0' shape=(None, 768) dtype=float32>)

In [ ]:
bert_outputs = bert_outputs[1]

In [ ]:
import tensorflow_addons as tfa
opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 50000, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)

In [ ]:
sentiment_drop = tf.keras.layers.Dropout(0.8)(bert_outputs)
sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(sentiment_drop)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)
sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [ ]:
sentiment_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_segment (InputLayer)      [(None, 64)]         0                                            
__________________________________________________________________________________________________
tf_bert_model_3 (TFBertModel)   ((None, 64, 768), (N 92186880    input_word_ids[0][0]             
                                                                 input_masks[0][0]          

## 모델 학습

In [ ]:
sentiment_model.fit(train_x, train_y, epochs=2, shuffle=True, batch_size=64, validation_split=0.2)

Epoch 1/2
682/682 [==============================] - 373s 547ms/step - loss: 0.0322 - accuracy: 0.9919 - val_loss: 0.5134 - val_accuracy: 0.8903
Epoch 2/2
682/682 [==============================] - 371s 543ms/step - loss: 0.0252 - accuracy: 0.9936 - val_loss: 0.4995 - val_accuracy: 0.8885


prediction

In [ ]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []
    
    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, pad_to_max_length=True)
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]
    
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

In [ ]:
test_set = predict_load_data(test)

100%|██████████| 13626/13626 [00:02<00:00, 5210.18it/s]


In [ ]:
test_set

[array([[   2,  790, 1460, ...,    1,    1,    1],
        [   2, 4147, 6507, ...,    1,    1,    1],
        [   2, 1789, 5760, ..., 6553, 6122,    3],
        ...,
        [   2, 3298, 7598, ...,    1,    1,    1],
        [   2,  517, 5545, ..., 6116, 2355,    3],
        [   2, 2801, 4655, ...,    1,    1,    1]]),
 array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 0],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [ ]:
preds = sentiment_model.predict(test_set).flatten()

In [ ]:
pd.DataFrame({'id':test.id, 'pred':preds}).to_csv('best_kobert_1epoch_prob.csv',index=False)

In [ ]:
res = []
for x in preds:
  if x >= 0.5:
    res.append(1)
  else:
    res.append(0)

In [ ]:
test.id
pd.DataFrame({'id':test.id, 'pred':res}).to_csv('best_kobert_1epoch_class.csv',index=False)

## 앙상블

In [ ]:
ko = pd.read_csv('/content/drive/My Drive/딥러닝/감성분석/best_kobert_3epoch_prob.csv')
ko_preds = ko.pred

In [ ]:
ls = pd.read_csv('/content/drive/My Drive/딥러닝/감성분석/best_LSTM_prob.csv')
ls_preds = ls.pred

In [ ]:
#cnn = pd.read_csv('/content/drive/My Drive/딥러닝/감성분석/best_cnn_prob.csv')
#cnn_preds = cnn.pred

In [ ]:
GM = (ko_preds * ls_preds) ** (1/2) # 기하 평균 Geometric Mean

In [ ]:
GM

0        0.012864
1        0.018670
2        0.005467
3        0.986147
4        0.781612
           ...   
13621    0.958113
13622    0.026518
13623    0.016730
13624    0.023093
13625    0.948271
Name: pred, Length: 13626, dtype: float64

In [ ]:
gm_res = []
for x in GM:
  if x >= 0.5:
    gm_res.append(1)
  else:
    gm_res.append(0)

In [ ]:
pd.DataFrame({'id':test.id, 'pred':gm_res}).to_csv('best_kobert_lstm_gmean3.csv',index=False)